In [188]:
#!pip install xgboost
import pandas as pd

df = pd.read_csv('../data/processed/credit_risk_dataset_processed.csv')
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,...,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_grade_encoded,cb_person_default_on_file_encoded
0,0.034483,0.009173,RENT,0.000000,PERSONAL,D,1.000000,0.595506,1,0.710843,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,1
1,0.017241,0.000934,OWN,0.121951,EDUCATION,B,0.014493,0.321348,0,0.120482,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0
2,0.086207,0.000934,MORTGAGE,0.024390,MEDICAL,C,0.144928,0.418539,1,0.686747,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0
3,0.051724,0.010257,RENT,0.097561,MEDICAL,C,1.000000,0.551124,1,0.638554,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0
4,0.068966,0.008406,RENT,0.195122,MEDICAL,C,1.000000,0.497191,1,0.662651,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1


In [189]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score
import time
from tabulate import tabulate

### 'cb_person_cred_hist_length',  
# Division des données
X = df[['person_age', 'person_income', 'person_emp_length', 'loan_amnt',  'loan_int_rate', 
 'loan_percent_income',  'loan_intent_DEBTCONSOLIDATION',
 'loan_intent_EDUCATION', 'loan_intent_HOMEIMPROVEMENT', 'loan_intent_MEDICAL',
 'loan_intent_PERSONAL',  'loan_intent_VENTURE', 'person_home_ownership_MORTGAGE',
 'person_home_ownership_OTHER', 'person_home_ownership_OWN', 'person_home_ownership_RENT',
 'loan_grade_encoded', 'cb_person_default_on_file_encoded']]

y = df['loan_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fonction pour entraîner et évaluer le modèle
def entrainer_evaluer_modele(nom, modele):
    debut = time.time()
    modele.fit(X_train, y_train)
    previsions = modele.predict(X_test)
    fin = time.time()
    matrice_confusion = confusion_matrix(y_test, previsions)
    
    return {
        'Modèle': nom,
        'Précision': precision_score(y_test, previsions),
        'Rappel': recall_score(y_test, previsions),
        'F1-score': f1_score(y_test, previsions),
        'Accuracy': accuracy_score(y_test, previsions),
        'Temps d\'exécution': fin - debut,
        'TP': matrice_confusion[1, 1],
        'FN': matrice_confusion[1, 0],
        'FP': matrice_confusion[0, 1],
        'TN': matrice_confusion[0, 0]
    }

# Dictionnaire des modèles
modeles = {
    'Régression logistique': LogisticRegression(max_iter=1000),
    'Arbre de décision': DecisionTreeClassifier(max_depth=10),
    'Forêt aléatoire': RandomForestClassifier(max_depth=20),
    'KNN': KNeighborsClassifier(n_neighbors=10),
    'Naive Bayes': GaussianNB(),
    'XGBoost': xgb.XGBClassifier(
        colsample_bytree=0.8, gamma=0.1, learning_rate=0.1,
        max_depth=6, n_estimators=200, subsample=1, random_state=42)
}

# Entraîner et collecter les résultats
resultats = [entrainer_evaluer_modele(nom, modele) for nom, modele in modeles.items()]

# Afficher les résultats
tableau = [[modele['Modèle'], 
            f"{modele['Précision']:.3f}", 
            f"{modele['Rappel']:.3f}", 
            f"{modele['F1-score']:.3f}", 
            f"{modele['Accuracy']:.3f}",
            f"{modele['Temps d\'exécution']:.2f}",
            f"{modele['TP']}", 
            f"{modele['FN']}", 
            f"{modele['FP']}", 
            f"{modele['TN']}"] 
           for modele in resultats]

print(tabulate(tableau, 
               headers=['Modèle', 'Précision', 'Rappel', 'F1-score', 'Accuracy', 'Temps d\'exécution', 'TP', 'FN', 'FP', 'TN'], 
               tablefmt='orgtbl'))


| Modèle                |   Précision |   Rappel |   F1-score |   Accuracy |   Temps d'exécution |   TP |   FN |   FP |   TN |
|-----------------------+-------------+----------+------------+------------+---------------------+------+------+------+------|
| Régression logistique |       0.727 |    0.489 |      0.585 |      0.846 |                0.07 |  707 |  738 |  265 | 4807 |
| Arbre de décision     |       0.865 |    0.621 |      0.723 |      0.894 |                0.82 |  897 |  548 |  140 | 4932 |
| Forêt aléatoire       |       0.913 |    0.665 |      0.769 |      0.912 |               20.47 |  961 |  484 |   92 | 4980 |
| KNN                   |       0.922 |    0.594 |      0.722 |      0.899 |                0.52 |  858 |  587 |   73 | 4999 |
| Naive Bayes           |       0.596 |    0.608 |      0.602 |      0.822 |                0.03 |  878 |  567 |  595 | 4477 |
| XGBoost               |       0.908 |    0.679 |      0.777 |      0.914 |                2.95 |  981 |  464 

Dans le contexte des prêts, le champ "loan_status" fait référence à l'état de remboursement du prêt. Dans ce cas précis :

- 0 (non-défaut) indique que l'emprunteur est à jour avec les paiements du prêt, c'est-à-dire qu'il n'y a pas de retards ou de défauts.
- 1 (défaut) indique que l'emprunteur ne paie pas le prêt conformément à l'accord, c'est-à-dire qu'il est en retard avec les paiements ou qu'il a déjà été considéré comme en défaut.


En d'autres termes :

- Non-défaut (0) : Le prêt est remboursé régulièrement et il n'y a pas de problèmes de paiement.
- Défaut (1) : Le prêt présente des problèmes de paiement, tels que des retards ou un manque de paiement.

Dans le contexte d'un modèle qui prédit des valeurs 0 ou 1 pour le champ "loan_status", voici ce que chaque sigle TP, TN, FP et FN représente :

TP (Vrai Positif) :
- Prévision : 1 (défaut)
- Réalité : 1 (défaut)
- Signification : Le modèle a prédit correctement que le prêt serait en défaut.


TN (Vrai Négatif) :
- Prévision : 0 (non-défaut)
- Réalité : 0 (non-défaut)
- Signification : Le modèle a prédit correctement que le prêt ne serait pas en défaut.


FP (Faux Positif) :
- Prévision : 1 (défaut)
- Réalité : 0 (non-défaut)
- Signification : Le modèle a prédit incorrectement que le prêt serait en défaut, alors qu'en réalité il ne l'était pas.

<span style="font-weight: bold;">FN (Faux Négatif): (Desirable moin)</span>
- Prévision : 0 (non-défaut)
- Réalité : 1 (défaut)
- Signification : Le modèle a prédit incorrectement que le prêt ne serait pas en défaut, alors qu'en réalité il l'était.
